In [1]:
# Kill all processess on GPU
!fuser -v /dev/nvidia* -k

# Libraries

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    %pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    %pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    %pip install sentencepiece protobuf "datasets>=4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    %pip install --no-deps unsloth
%pip install trl==0.19.1 # Fix error: ImportError: cannot import name 'ConstantLengthDataset' from 'trl.trainer.utils'

In [3]:
import os
import functools
import gc
import json
import random
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from typing import Optional, Literal, Union, List
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
from transformers import default_data_collator, AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from huggingface_hub import snapshot_download, create_repo, upload_folder
from safetensors.torch import load_file, save_file
from datetime import datetime
from pprint import pprint

import cuml
from cuml.decomposition import PCA as cuPCA
from cuml.manifold import UMAP as cuUMAP

In [4]:
def download_hf_model(
        repo_id: str, 
        checkpoint: Optional[int], 
        max_checkpoints: str = 10_000, 
        checkpoint_steps: str = 25,
    ):
    local_dir = repo_id.split('/')[-1]
    ignore_checkpoints = None
    
    if checkpoint is not None:
        ignore_checkpoints = [f'checkpoint-{i}/*' for i in range(0, max_checkpoints, checkpoint_steps) if i != checkpoint]

    snapshot_download(
        repo_id=repo_id,
        local_dir=local_dir,
        ignore_patterns=ignore_checkpoints,
    )

    checkpoint_dir = None
    if checkpoint is not None:
        checkpoint_dir = os.path.join(local_dir, f'checkpoint-{checkpoint}')
    return local_dir, checkpoint_dir

def check_loss_and_grad_norm(
    model, 
    tokenizer, 
    prompt="Paris is the capital of",
):
    # Set model to training mode
    model.train()

    # Zero gradients manually
    for p in model.parameters():
        p.grad = None

    # Forward pass
    inputs = tokenizer(
        prompt, 
        return_tensors='pt',
    ).to(next(model.parameters()).device)

    outputs = model(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        labels=inputs['input_ids'],
        use_cache=False, # Disable cache to not conflict with gradient checkpointing
    )
    if isinstance(outputs, tuple):
        outputs = outputs[0]
    print("Loss:", outputs.loss)

    # Backward pass
    if outputs.loss.grad_fn is None:
        print("Gradient norm:", None)
        return

    outputs.loss.backward()

    # Compute gradient norm
    grad_norm = 0.0
    for n, p in model.named_parameters():
        if p.requires_grad:
            p_grad_norm = p.grad.data.norm(2)
            grad_norm += p_grad_norm.item() ** 2
    grad_norm = grad_norm ** 0.5

    print("Gradient norm:", grad_norm)

def check_parameter(n, p):
    print(f"- {'name':<8}:", n)
    print(f"- {'device':<8}:", p.device)
    print(f"- {'dtype':<8}:", p.dtype)
    print(f"- {'mean':<8}:", p.mean().item())
    print(f"- {'min':<8}:", p.min().item())
    print(f"- {'max':<8}:", p.max().item())

def check_lora_parameters(model, prefix=None):
    prefix = 'lora.' + prefix if prefix != None else 'lora'
    for n, p in model.named_parameters():
        if prefix in n:
            check_parameter(n, p)
            break

@torch.no_grad()
def generate_text(model, tokenizer, prompt, max_new_tokens=32, device=None, skip_special_tokens=True):
    device = device or next(model.parameters()).device
    model.eval()
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=max_new_tokens)
    model.train()
    return tokenizer.decode(outputs[0], skip_special_tokens=skip_special_tokens)

def get_task_and_lang_from_repo_id(repo_id: str):
    task, lang, _ = repo_id.split('B-')[-1].split('K-')[0].split('-')
    return task, lang

def load_hf_dataset_from_lora(
    lora_repo_id: str,
    train_size: int = 5000,
    test_size: int = 1000,
):
    # Get task and language
    task, lang = get_task_and_lang_from_repo_id(lora_repo_id)

    # Set up Hugging Face configuration
    data_id_map = {
        'wikipedia': 'wikimedia/wikipedia',
        'gsm8k': 'openai/gsm8k',
    }
    data_id = data_id_map[task]
    data_dir = f'20231101.{lang}' if task == 'wikipedia' else 'main'
    
    # Load dataset using streaming
    dataset_stream = load_dataset(data_id, data_dir=data_dir, split='train', streaming=True)

    # Manually take train_size + test_size samples
    total_size = train_size + test_size
    sliced_data = []
    for i, example in enumerate(dataset_stream):
        if i >= total_size:
            break
        sliced_data.append(example)

    # Convert to regular in-memory dataset
    dataset = Dataset.from_list(sliced_data)
    
    return dataset

def load_hf_dataset(
    lang, 
    task,
    split='train',
    train_size = 5000,
    test_size = 1000,
):
    # Set up Hugging Face configuration
    data_id_map = {
        'wikipedia': 'wikimedia/wikipedia',
        'gsm8k': 'openai/gsm8k',
    }
    data_id = data_id_map[task]
    data_dir = f'20231101.{lang}' if task == 'wikipedia' else 'main'

    # Use streaming
    dataset_stream = load_dataset(data_id, data_dir=data_dir, split=split, streaming=True)

    # Manually take train_size + test_size samples
    total_size = train_size + test_size
    sliced_data = []
    for i, example in enumerate(dataset_stream):
        if i >= total_size:
            break
        sliced_data.append(example)

    # Convert to regular in-memory dataset
    dataset = Dataset.from_list(sliced_data)
    
    return dataset

def compute_grad_norm(params):
    grad_norm = 0.0
    for p in params:
        p_grad_norm = p.grad.data.norm(2)
        grad_norm += p_grad_norm.item() ** 2
    grad_norm = grad_norm ** 0.5
    return grad_norm

def compute_named_grad_norm(named_params):
    grad_norm = 0.0
    for n, p in named_params.items():
        if p.grad is not None:
            p_grad_norm = p.grad.data.norm(2)
            grad_norm += p_grad_norm.item() ** 2
        else:
            print(f"[WARN] No gradient for {n}")
    grad_norm = grad_norm ** 0.5
    return grad_norm

def format_float(v):
    if abs(v) < 0.0001 or abs(v) >= 10000:
        return f"{v:.4e}"
    else:
        return f"{v:.4f}"

# Configurations

In [5]:
# Project configuration
seed = 69
device = 'auto'

# Data configuration
hf_data_id = 'alxxtexxr/Nero-XLT-Dataset'
hf_data_dir = 'gsm8k_en_5K_1K_1K_512'

# Training configuration
batch_size = 4
num_epochs = 1
max_global_steps = None
grad_accumulation_steps = 2
clip_grad_norm = 1.0
lr = 2e-4
warmup_ratio = 0.1
# num_warmup_steps = 100
checkpoint_steps = 25
push_to_hf = False
generate_steps = 25
sample_prompt = '102452 + 102453 ='
distance_fn = 'euclidean'

# Model configurations
model_configs = {
    # L1T1 (Source Language - Source Task)
    'L1T1': {
        'hf_lora_id': 'alxxtexxr/L3.1-8B-wikipedia-en-5K-LoRA-v20250630122650',
        'checkpoint': 650,
    },

    # L2T1 (Target Language - Source Task)
    'L2T1': {
        'hf_lora_id': 'alxxtexxr/L3.1-8B-wikipedia-ja-5K-LoRA-v20250728141629',
        'checkpoint': 650,
    },

    # L1T2 (Source Language - Target Task)
    # 'L1T2': {
    #     'hf_lora_id': 'alxxtexxr/L3.1-8B-gsm8k-en-5K-LoRA-v20250701060457',
    #     'checkpoint': 1875,
    # },
}

for key, config in model_configs.items():
    _, lora_dir = download_hf_model(config['hf_lora_id'], config['checkpoint'])
    model_configs[key]['lora_dir'] = lora_dir
    model_configs[key]['lora_path'] = os.path.join(lora_dir, 'adapter_model.safetensors')
    model_configs[key]['lora_config'] = LoraConfig.from_pretrained(lora_dir)

print("Model configurations:",)
for key, config in model_configs.items():
    print(f"- {key}:")
    for config_name, config_value in config.items():
        if config_name == 'lora_config':
            continue
        print(f"{'-':>3} {config_name:<10}: {config_value}")
print()

assert (
    model_configs['L1T1']['lora_config'].base_model_name_or_path == 
    model_configs['L2T1']['lora_config'].base_model_name_or_path
), "Base models must be the same"
base_model_name = model_configs['L1T1']['lora_config'].base_model_name_or_path
print(f"Base model name: {base_model_name}")

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

Model configurations:
- L1T1:
  - hf_lora_id: alxxtexxr/L3.1-8B-wikipedia-en-5K-LoRA-v20250630122650
  - checkpoint: 650
  - lora_dir  : L3.1-8B-wikipedia-en-5K-LoRA-v20250630122650/checkpoint-650
  - lora_path : L3.1-8B-wikipedia-en-5K-LoRA-v20250630122650/checkpoint-650/adapter_model.safetensors
- L2T1:
  - hf_lora_id: alxxtexxr/L3.1-8B-wikipedia-ja-5K-LoRA-v20250728141629
  - checkpoint: 650
  - lora_dir  : L3.1-8B-wikipedia-ja-5K-LoRA-v20250728141629/checkpoint-650
  - lora_path : L3.1-8B-wikipedia-ja-5K-LoRA-v20250728141629/checkpoint-650/adapter_model.safetensors

Base model name: unsloth/meta-llama-3.1-8b-unsloth-bnb-4bit


In [6]:
# Hugging Face configuration
hf_nero_id = None
resume_step = 0

if hf_nero_id is not None and resume_step > 0:
    print(f"[INFO] Downloading Nero checkpoint at step {resume_step} from Hugging Face repository:", hf_nero_id)
    nero_dir, _ = download_hf_model(hf_nero_id, resume_step)
    print(f"[INFO] Nero checkpoint downloaded successfully!")
else:
    hf_username = 'alxxtexxr'
    nero_dir = f'L3.1-8B-{hf_data_dir.replace("_", "-")}-Nero-{distance_fn}-v{datetime.now().strftime("%Y%m%d%H%M%S")}'
    print(f"[INFO] Creating Nero directory:", nero_dir)
    hf_nero_id = f'{hf_username}/{nero_dir}'
    os.makedirs(nero_dir, exist_ok=True)
    print(f"[INFO] Nero directory created!")

[INFO] Creating Nero directory: L3.1-8B-gsm8k-en-5K-1K-1K-512-Nero-euclidean-v20250820163350
[INFO] Nero directory created!


# Model

In [7]:
def flatten_emb(emb):
    return emb.reshape(-1, emb.shape[-1])

class NeroLayer(nn.Module):
    def __init__(self, base_layer,
                 
                 # LoRA parameters
                 L1T1_lora_params, 
                 L2T1_lora_params,

                 # Nero parameters
                 nero_params,
                 train_L2T2_nero = False,
                 eval_L2T2_nero = False,
                 return_L2T2_nero_loss = False,
                 
                 # Distance function parameters  
                 distance_fn='euclidean',
                 distance_fn_pairwise=True,
                 
                 debug=False,
                 module_name=None,
                 ):
        super().__init__()

        self.base_layer = base_layer
        self.device = base_layer.weight.device
        self.train_L2T2_nero = train_L2T2_nero
        self.eval_L2T2_nero = eval_L2T2_nero
        self.return_L2T2_nero_loss = return_L2T2_nero_loss
        
        self.distance_fn = distance_fn
        self.distance_fn_pairwise = distance_fn_pairwise

        self.debug = debug
        self.module_name = module_name

        # Extract input and output features from the base layer
        in_features = getattr(base_layer, 'in_features', None)
        out_features = getattr(base_layer, 'out_features', None)

        if in_features is None or out_features is None:
            raise ValueError(f"Cannot determine in_features or out_features from {base_layer}.")

        # Initialize LoRA and Nero layers
        self.lora = nn.ModuleDict({
            'L1T1': self._init_lora_layer(in_features, out_features, **L1T1_lora_params, device=self.device),
            'L2T1': self._init_lora_layer(in_features, out_features, **L2T1_lora_params, device=self.device),
        })
        self.nero = nn.ModuleDict({
            'L1T2': self._init_nero_layer(out_features, **nero_params, device=self.device),
            'L2T2': self._init_nero_layer(out_features, **nero_params, device=self.device),
        })
        
    def _init_lora_layer(self, in_features, out_features, rank, alpha, dropout=0.0, bias=True, use_rslora=False, device=None):
        scaling = alpha / math.sqrt(rank) if use_rslora else alpha / rank
        dropout_layer = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        layer = nn.ModuleDict({
            'A': nn.Linear(in_features, rank, bias=bias, device=device),
            'B': nn.Linear(rank, out_features, bias=bias, device=device),
            'dropout': dropout_layer
        })
        layer.scaling = scaling
        layer.bias_flag = bias
        nn.init.normal_(layer['A'].weight, 0.0, 1 / math.sqrt(rank))
        nn.init.zeros_(layer['B'].weight)
        return layer

    def _init_nero_layer(self, features, rank, bias=True, device=None):
        layer = nn.ModuleDict({
            'A': nn.Linear(features, rank, bias=bias, device=device),
            'B': nn.Linear(rank, features, bias=bias, device=device),
        })
        layer.bias_flag = bias
        nn.init.normal_(layer['A'].weight, 0.0, 1 / math.sqrt(rank))
        nn.init.zeros_(layer['B'].weight)
        return layer
    
    def _distance_fn(self, a, b, huber_delta=1.0, eps=1e-8,
                    soft_radius_a=0.1, soft_radius_b=0.1, soft_num_samples=5, chunk_size=64):
        def huber_loss(diff, delta):
            mask = diff.abs() < delta
            return torch.where(mask, 0.5 * diff**2, delta * (diff.abs() - 0.5 * delta))

        if self.distance_fn == 'soft_euclidean':
            if self.distance_fn_pairwise:
                # Ensure we always have (N, D) and (M, D) for chunking
                if a.dim() == 3:
                    # Already expanded to (N, M, D) → collapse back
                    N, M, D = a.shape
                    a_base = a[:, 0, :]  # take first along M axis
                    b_base = b[0, :, :]  # take first along N axis
                else:
                    N, D = a.shape
                    M = b.shape[0]
                    a_base, b_base = a, b

                dist_list = []
                for i in range(0, N, chunk_size):
                    a_chunk = a_base[i:i+chunk_size]  # (chunk, D)
                    a_exp = a_chunk.unsqueeze(1).expand(-1, M, D)
                    b_exp = b_base.unsqueeze(0).expand(a_chunk.shape[0], -1, D)

                    a_pert = a_exp.unsqueeze(2) + torch.randn(
                        a_exp.shape[0], M, soft_num_samples, D, device=a.device
                    ) * soft_radius_a
                    b_pert = b_exp.unsqueeze(2) + torch.randn(
                        a_exp.shape[0], M, soft_num_samples, D, device=b.device
                    ) * soft_radius_b

                    diff = a_pert.unsqueeze(3) - b_pert.unsqueeze(2)  # (chunk, M, num_a, num_b, D)
                    dist = torch.sqrt((diff**2).sum(dim=-1) + eps)
                    min_dist = dist.min(dim=2)[0].min(dim=-1)[0]  # (chunk, M)
                    dist_list.append(min_dist)
                return torch.cat(dist_list, dim=0)

        # Normal distances
        if self.distance_fn_pairwise:
            a_exp = a.unsqueeze(1)
            b_exp = b.unsqueeze(0)
            diff = a_exp - b_exp
        else:
            diff = a - b

        if self.distance_fn == 'euclidean':
            return torch.sqrt((diff**2).sum(dim=-1) + eps)
        elif self.distance_fn == 'squared_euclidean':
            return (diff**2).sum(dim=-1)
        elif self.distance_fn == 'huber':
            return huber_loss(diff, huber_delta).sum(dim=-1)
        elif self.distance_fn == 'cosine':
            a_norm = F.normalize(a_exp if self.distance_fn_pairwise else a, dim=-1, eps=eps)
            b_norm = F.normalize(b_exp if self.distance_fn_pairwise else b, dim=-1, eps=eps)
            return 1 - (a_norm * b_norm).sum(dim=-1)
        else:
            raise ValueError(f"Unsupported distance function: {self.distance_fn}")

    def _rbf_random_fourier_features(self, X, D_rff=512, gamma=1e-3):
        """
        X: (N, d)
        Returns: (N, D_rff) differentiable RBF approximation
        """
        N, d = X.shape
        W = torch.randn(d, D_rff, device=X.device) * (2 * gamma)**0.5
        b = 2 * torch.pi * torch.rand(D_rff, device=X.device)
        Z = torch.sqrt(torch.tensor(2.0 / D_rff, device=X.device)) * torch.cos(X @ W + b)
        return Z
        
    def forward(self, x):
        # ================================================================
        # BASE LAYER
        # ================================================================
        base_out = self.base_layer(x)

        requires_conversion = not torch.is_autocast_enabled()
        if requires_conversion:
            assert self.lora.L1T1.A.weight.dtype == self.lora.L2T1.A.weight.dtype
            x = x.to(self.lora.L1T1.A.weight.dtype)

        if not self.eval_L2T2_nero:
            # ================================================================
            # L1T1 LoRA LAYER
            # ================================================================
            # L1T1_lora_out = self.lora.L1T1.B(self.lora.L1T1.A(self.lora.L1T1.dropout(x))) #* self.lora.L1T1.scaling
            L1T1_lora_dropout_out = self.lora.L1T1.dropout(x)
            L1T1_lora_A_out = self.lora.L1T1.A(L1T1_lora_dropout_out)
            L1T1_lora_B_out = self.lora.L1T1.B(L1T1_lora_A_out)
            # L1T1_lora_out = L1T1_lora_B_out * self.lora.L1T1.scaling
            L1T1_lora_out = L1T1_lora_B_out

            # ================================================================
            # L1T2 Nero LAYER
            # ================================================================
            # L1T2_nero_out = self.nero.L1T2.B(F.relu(self.nero.L1T2.A(L1T1_lora_out)))
            L1T2_nero_A_out = self.nero.L1T2.A(L1T1_lora_out)
            L1T2_nero_relu_out = F.relu(L1T2_nero_A_out)
            L1T2_nero_B_out = self.nero.L1T2.B(L1T2_nero_relu_out)
            L1T2_nero_out = L1T2_nero_B_out * self.lora.L1T1.scaling
            if requires_conversion:
                L1T2_nero_out = L1T2_nero_out.to(base_out.dtype)
            output = base_out + L1T2_nero_out

        if self.eval_L2T2_nero or self.train_L2T2_nero:
            # ================================================================
            # L2T1 LoRA LAYER
            # ================================================================
            # L2T1_lora_out = self.lora.L2T1.B(self.lora.L2T1.A(self.lora.L2T1.dropout(x))) #* self.lora.L2T1.scaling
            L2T1_lora_dropout_out = self.lora.L2T1.dropout(x)
            L2T1_lora_A_out = self.lora.L2T1.A(L2T1_lora_dropout_out)
            L2T1_lora_B_out = self.lora.L2T1.B(L2T1_lora_A_out)
            # L2T1_lora_out = L2T1_lora_B_out * self.lora.L2T1.scaling
            L2T1_lora_out = L2T1_lora_B_out

            # ================================================================
            # L2T2 Nero LAYER
            # ================================================================
            # L2T2_nero_out = self.nero.L2T2.B(F.relu(self.nero.L2T2.A(L2T1_lora_out)))
            L2T2_nero_A_out = self.nero.L2T2.A(L2T1_lora_out)
            L2T2_nero_relu_out = F.relu(L2T2_nero_A_out)
            L2T2_nero_B_out = self.nero.L2T2.B(L2T2_nero_relu_out)
            L2T2_nero_out = L2T2_nero_B_out * self.lora.L2T1.scaling
            if requires_conversion:
                L2T2_nero_out = L2T2_nero_out.to(base_out.dtype)
            
            # ================================================================
            # FLATTEN AND CONCATENATE EMBEDDINGS
            # ================================================================
            embeddings_list = [
                flatten_emb(L1T1_lora_out),
                flatten_emb(L1T2_nero_out),
                flatten_emb(L2T1_lora_out),
                flatten_emb(L2T2_nero_out),
            ]
            
            all_embeddings = torch.cat(embeddings_list, dim=0)
            
            # ================================================================
            # DIFFERENTIAL PCA VIA SVD
            # ================================================================
            pca_dim = 512
            mean = all_embeddings.mean(dim=0, keepdim=True)
            X_centered = all_embeddings - mean
            U, S, Vh = torch.linalg.svd(X_centered, full_matrices=False)
            all_embeddings_pca = X_centered @ Vh[:pca_dim, :].T  # (N, pca_dim)
            
            # ================================================================
            # DIFFERENTIAL RFF
            # ================================================================
            shared_space = self._rbf_random_fourier_features(all_embeddings_pca, D_rff=512)
            
            # ================================================================
            # SEPARATE EMBEDDINGS
            # ================================================================
            sizes = [e.shape[0] for e in embeddings_list]
            indices = np.cumsum([0] + sizes)
            shared_domains = [shared_space[indices[i]:indices[i+1]] for i in range(4)]

            # ================================================================
            # LOSS
            # ================================================================
            # Cross-language distances for the same task: 
            #   Task 1: distance(L1T1, L2T1) 
            #   Task 2: distance(L1T2, L2T2)
            LxT1_distance = self._distance_fn(shared_domains[0].float(), shared_domains[2].float())
            LxT2_distance = self._distance_fn(shared_domains[1].float(), shared_domains[3].float()) # requires grad

            # Cross-task distances for the same language: 
            #   Language 1: distance(L1T1, L1T2)
            #   Language 2: distance(L2T1, L2T2)
            L1Tx_distance = self._distance_fn(shared_domains[0].float(), shared_domains[1].float())
            L2Tx_distance = self._distance_fn(shared_domains[2].float(), shared_domains[3].float()) # requires grad

            # Distance constraints (enforced via MSE loss):
            #   Cross-language distances should match across tasks:
            #       distance(L1T1, L2T1) ≈ distance(L1T2, L2T2)
            #   Cross-task distances should match across languages:
            #       distance(L1T1, L1T2) ≈ distance(L2T1, L2T2)
            L2T2_nero_loss = F.mse_loss(LxT2_distance, LxT1_distance) + F.mse_loss(L2Tx_distance, L1Tx_distance)

            # ================================================================
            # OUTPUT
            # ================================================================
            # If `output` is None due to being in evaluate L2T2 Nero mode, 
            # compute and return the L2T2 Nero-injected output
            if output is None:
                output = base_out + L2T2_nero_loss
                return output
            
            # If enabled, return both the L1T2 Nero–injected output and the L2T2 Nero loss
            if self.return_L2T2_nero_loss:
                return output, L2T2_nero_loss

        # Return only the L1T2 Nero-injected output
        return output

    def load_lora_params(self, mode: Literal['L1T1', 'L2T1'], state_dict, prefix: str):
        self.lora[mode].A.weight.data = state_dict[f'{prefix}.lora_A.weight'].to(self.device)
        self.lora[mode].B.weight.data = state_dict[f'{prefix}.lora_B.weight'].to(self.device)
        if self.lora[mode].bias_flag:
            self.lora[mode].A.bias.data = state_dict[f'{prefix}.lora_A.bias'].to(self.device)
            self.lora[mode].B.bias.data = state_dict[f'{prefix}.lora_B.bias'].to(self.device)

    def load_nero_params(self, mode: Literal['L1T2', 'L2T2'], state_dict, prefix: str):
        self.nero[mode].A.weight.data = state_dict[f'{prefix}.nero.{mode}.A.weight'].to(self.device)
        self.nero[mode].B.weight.data = state_dict[f'{prefix}.nero.{mode}.B.weight'].to(self.device)
        if self.nero[mode].bias_flag:
            self.nero[mode].A.bias.data = state_dict[f'{prefix}.nero.{mode}.A.bias'].to(self.device)
            self.nero[mode].B.bias.data = state_dict[f'{prefix}.nero.{mode}.B.bias'].to(self.device)
    
    def reinitialize_nero_params(self, modes: Union[List[Literal['L1T2', 'L2T2']], Literal['L1T2', 'L2T2', 'all']] = 'all'):
        """
        Reinitialize the parameters of Nero layers.
        
        Args:
            modes: Which Nero layers to reinitialize. Can be:
                - 'all': Reinitialize both L1T2 and L2T2 (default)
                - 'L1T2': Reinitialize only L1T2 layer
                - 'L2T2': Reinitialize only L2T2 layer  
                - List of modes: e.g., ['L1T2', 'L2T2'] or ['L1T2']
        """
        # Handle different input formats
        if modes == 'all':
            target_modes = ['L1T2', 'L2T2']
        elif isinstance(modes, str):
            target_modes = [modes]
        else:
            target_modes = modes
        
        # Validate modes
        valid_modes = {'L1T2', 'L2T2'}
        for mode in target_modes:
            if mode not in valid_modes:
                raise ValueError(f"Invalid mode '{mode}'. Must be one of {valid_modes}")
        
        # Reinitialize specified Nero layers
        for mode in target_modes:
            if mode in self.nero:
                nero_layer = self.nero[mode]
                
                # Get the rank from the A layer's output features
                rank = nero_layer['A'].out_features
                
                # Reinitialize A layer weights: normal distribution with std = 1/sqrt(rank)
                nn.init.normal_(nero_layer['A'].weight, mean=0.0, std=1.0 / math.sqrt(rank))
                
                # Reinitialize B layer weights: zeros
                nn.init.zeros_(nero_layer['B'].weight)
                
                # Reinitialize biases if they exist
                if nero_layer.bias_flag:
                    if nero_layer['A'].bias is not None:
                        nn.init.zeros_(nero_layer['A'].bias)
                    if nero_layer['B'].bias is not None:
                        nn.init.zeros_(nero_layer['B'].bias)
                
                if self.debug:
                    print(f"Reinitialized Nero layer {mode} (rank={rank})")
        
        if self.debug:
            print(f"Nero parameter reinitialization complete for modes: {target_modes}")
        
class NeroModel(nn.Module):
    def __init__(self, 
                 base_model: nn.Module, 
                 L1T1_lora_config: LoraConfig, 
                 L2T1_lora_config: LoraConfig, 
                 train_L2T2_nero=True,
                 eval_L2T2_nero=False,
                 return_L2T2_nero_loss=False,
                 distance_fn='euclidean',
                 distance_fn_pairwise=True,
                 debug: bool=False,
                ):
        super().__init__()
        self.base_model = base_model
        # assert train_L2T2_nero != eval_L2T2_nero
        self.train_L2T2_nero = train_L2T2_nero
        self.eval_L2T2_nero = eval_L2T2_nero
        self.return_L2T2_nero_loss = return_L2T2_nero_loss
        
        self.distance_fn = distance_fn
        self.distance_fn_pairwise = distance_fn_pairwise

        self.debug = debug

        # Wrap target layers with NeroLayer
        self.nero_layers = nn.ModuleDict()
        self._wrap_target_layers(L1T1_lora_config, L2T1_lora_config)
        
    def _wrap_target_layers(self, L1T1_lora_config, L2T1_lora_config):
        assert L1T1_lora_config.target_modules == L2T1_lora_config.target_modules, "[ERROR] L1T1 and L2T1 LoRA configurations must have the same target modules."

        for module_name, module in self.base_model.named_modules():
            if isinstance(module, NeroLayer):
                # Convert module name format and store reference
                module_name = module_name.rsplit('model.', 1)[-1]
                module_name = module_name.replace('.', '__DOT__')
                self.nero_layers[module_name] = module
                continue

            if any(module_name.endswith(target_module) for target_module in L1T1_lora_config.target_modules) and isinstance(module, nn.Linear):    
                parent_module, child_name = self._get_parent_module(module_name)
                nero_layer = NeroLayer(
                    base_layer=module,
                    train_L2T2_nero=self.train_L2T2_nero,
                    eval_L2T2_nero=self.eval_L2T2_nero,
                    return_L2T2_nero_loss=self.return_L2T2_nero_loss,

                    # L1T1 LoRA parameters
                    L1T1_lora_params={
                        'rank': L1T1_lora_config.r, 
                        'alpha': L1T1_lora_config.lora_alpha, 
                        'dropout': L1T1_lora_config.lora_dropout,
                        'bias': L1T1_lora_config.lora_bias,
                        'use_rslora': L1T1_lora_config.use_rslora,
                    },
                
                    # L2T1 LoRA parameters
                    L2T1_lora_params={
                        'rank': L2T1_lora_config.r, 
                        'alpha': L2T1_lora_config.lora_alpha, 
                        'dropout': L2T1_lora_config.lora_dropout,
                        'bias': L2T1_lora_config.lora_bias,
                        'use_rslora': L2T1_lora_config.use_rslora,
                    },

                    # Nero parameters (for temporary, use L1T1 LoRA parameters)
                    nero_params={
                        'rank': L1T1_lora_config.r, 
                        'bias': L1T1_lora_config.lora_bias,
                    },

                    # Distance function parameters
                    distance_fn=self.distance_fn,
                    distance_fn_pairwise=self.distance_fn_pairwise,

                    debug=self.debug,
                    module_name=module_name,
                )
                setattr(parent_module, child_name, nero_layer)

                # Store LoRA layers for weight loading
                module_name = module_name.rsplit('model.', 1)[-1]
                module_name = module_name.replace('.', '__DOT__')
                self.nero_layers[module_name] = nero_layer
    
    def _get_parent_module(self, module_name):
        parts = module_name.split('.')
        parent_module = self.base_model
        for part in parts[:-1]:
            parent_module = getattr(parent_module, part)
        return parent_module, parts[-1]
    
    def set_train_L2T2_nero(self, train_L2T2_nero: bool=True, verbose: bool=False):
        self.train_L2T2_nero = train_L2T2_nero
        for layer in self.nero_layers.values():
            layer.train_L2T2_nero = train_L2T2_nero
        
        if train_L2T2_nero:
            self.set_return_L2T2_nero_loss(True)
            self.freeze_all_except_L2T2_nero()
        else:
            self.set_return_L2T2_nero_loss(False)
            self.freeze_all_except_L1T2_nero()

        if verbose:
            print(f"[INFO] Train L2T2 Nero set to '{train_L2T2_nero}'.")

    def set_eval_L2T2_nero(self, eval_L2T2_nero: bool=True, verbose: bool=False):
        self.eval_L2T2_nero = eval_L2T2_nero
        for layer in self.nero_layers.values():
            layer.eval_L2T2_nero = eval_L2T2_nero
        
        if eval_L2T2_nero:
            self.set_return_L2T2_nero_loss(False)
            self.freeze_all()
        else:
            self.set_return_L2T2_nero_loss(True)
            self.freeze_all_except_L2T2_nero()

        if verbose:
            print(f"[INFO] Evaluate L2T2 Nero set to '{eval_L2T2_nero}'.")
    
    def set_return_L2T2_nero_loss(self, return_L2T2_nero_loss: bool=True, verbose: bool=False):
        self.return_L2T2_nero_loss = return_L2T2_nero_loss
        for layer in self.nero_layers.values():
            layer.return_L2T2_nero_loss = return_L2T2_nero_loss
        
        if verbose:
            print(f"[INFO] Return L2T2 Nero loss set to '{return_L2T2_nero_loss}'.")
    
    def set_distance_fn(self, distance_fn: str, pairwise: bool, verbose: bool=False):
        self.distance_fn = distance_fn
        self.distance_fn_pairwise = pairwise
        for layer in self.nero_layers.values():
            layer.distance_fn = distance_fn
            layer.distance_fn_pairwise = pairwise
        
        if verbose:
            print(f"[INFO] Distance function set to '{distance_fn}' with pairwise to {pairwise}.")
    
    def freeze_all(self, verbose: bool=False):
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        if verbose:
            print("[INFO] All layers are frozen!")

    def freeze_all_except_L1T2_nero(self, verbose=False):
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        for nero_layer in self.nero_layers.values():
            for param_name, param in nero_layer.named_parameters():
                if 'nero.L1T2.A' in param_name or 'nero.L1T2.B' in param_name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False
        
        if verbose:
            print("[INFO] All layers are frozen except L1T2 Nero layers!")
    
    def freeze_all_except_L2T2_nero(self, verbose=False):
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        for nero_layer in self.nero_layers.values():
            for param_name, param in nero_layer.named_parameters():
                if 'nero.L2T2.A' in param_name or 'nero.L2T2.B' in param_name:
                    param.requires_grad = True
                else:
                    param.requires_grad = False
        
        if verbose:
            print("[INFO] All layers are frozen except L2T2 Nero layers!")
    
    def unfreeze_all(self):
        for param in self.base_model.parameters():
            param.requires_grad = True
        
        for nero_layer in self.nero_layers.values():
            for param in nero_layer.parameters():
                param.requires_grad = True
        
        print("[INFO] All layers are unfrozen!")
    
    def load_lora_params(self, mode: Literal['L1T1', 'L2T1'], lora_path: str):
        if not os.path.exists(lora_path):
            raise FileNotFoundError("[ERROR] LoRA file not found:", lora_path)
        
        if lora_path.endswith('.safetensors'):
            state_dict = load_file(lora_path)
        else:
            state_dict = torch.load(lora_path, map_location='cpu')

        prefix = list(state_dict.keys())[0].rsplit('model.', 1)[0] + 'model.'

        for nero_layer_name, nero_layer in self.nero_layers.items():
            nero_layer_name = nero_layer_name.replace('__DOT__', '.')
            nero_layer_name = prefix + nero_layer_name
            if f'{nero_layer_name}.lora_A.weight' in state_dict and f'{nero_layer_name}.lora_B.weight' in state_dict:
                nero_layer.load_lora_params(mode, state_dict, nero_layer_name)
            else:
                # TODO: Print warning message
                pass

        print(f"[INFO] {mode} LoRA parameters loaded successfully!")
    
    def load_nero_params(self, mode: Literal['L1T2', 'L2T2'], nero_path):
        if not os.path.exists(nero_path):
            raise FileNotFoundError("[ERROR] Nero file not found:", nero_path)
        
        if nero_path.endswith('.safetensors'):
            state_dict = load_file(nero_path) # assuming .safetensors
        else:
            state_dict = torch.load(nero_path, map_location='cpu')

        prefix = list(state_dict.keys())[0].rsplit('model.', 1)[0] + 'model.'

        for nero_layer_name, nero_layer in self.nero_layers.items():
            nero_layer_name = nero_layer_name.replace('__DOT__', '.')
            nero_layer_name = prefix + nero_layer_name
            if f'{nero_layer_name}.nero.{mode}.A.weight' in state_dict and f'{nero_layer_name}.nero.{mode}.B.weight' in state_dict:
                nero_layer.load_nero_params(mode, state_dict, nero_layer_name)
            else:
                # TODO: Print warning message
                pass

        print(f"[INFO] {mode} Nero parameters loaded successfully!")
    
    def reinitialize_nero_params(self, modes: Union[List[Literal['L1T2', 'L2T2']], Literal['L1T2', 'L2T2', 'all']] = 'all', verbose: bool = False):
        """
        Reinitialize the parameters of Nero layers across all wrapped layers in the model.
        
        Args:
            modes: Which Nero layers to reinitialize. Can be:
                - 'all': Reinitialize both L1T2 and L2T2 (default)
                - 'L1T2': Reinitialize only L1T2 layers
                - 'L2T2': Reinitialize only L2T2 layers  
                - List of modes: e.g., ['L1T2', 'L2T2'] or ['L1T2']
            verbose: Whether to print detailed information about reinitialization
        """
        # Handle different input formats
        if modes == 'all':
            target_modes = ['L1T2', 'L2T2']
        elif isinstance(modes, str):
            target_modes = [modes]
        else:
            target_modes = modes
        
        # Validate modes
        valid_modes = {'L1T2', 'L2T2'}
        for mode in target_modes:
            if mode not in valid_modes:
                raise ValueError(f"Invalid mode '{mode}'. Must be one of {valid_modes}")
        
        # Reinitialize parameters for all Nero layers
        reinitialized_count = 0
        for layer_name, nero_layer in self.nero_layers.items():
            nero_layer.reinitialize_nero_params(modes)
            reinitialized_count += 1
            
            if verbose:
                print(f"Reinitialized Nero parameters for layer: {layer_name.replace('__DOT__', '.')}")
        
        if verbose or self.debug:
            print(f"[INFO] Reinitialized Nero parameters for {reinitialized_count} layers, modes: {target_modes}")
    
    def forward(self, *args, **kwargs):
        if self.train_L2T2_nero:
            L2T2_nero_losses = []
            
            def _hook_fn(layer_name, module, _in, _out):
                assert isinstance(_out, tuple) and len(_out) == 2
                layer_out, L2T2_nero_layer_loss = _out
                L2T2_nero_losses.append(L2T2_nero_layer_loss)
                return layer_out # Return only layer_out to avoid breaking model flow

            # Register hooks to extract hidden_out during forward pass
            hooks = []
            for layer_name, layer in self.nero_layers.items():
                hook = layer.register_forward_hook(functools.partial(_hook_fn, layer_name))
                hooks.append(hook)
        
            try:
                outputs = self.base_model(*args, **kwargs)
            finally:
                # Remove hooks after forward pass, ensuring it's done even if an error occurs
                for hook in hooks:
                    hook.remove()

            L2T2_nero_loss = torch.stack(L2T2_nero_losses).mean()
            
            if self.return_L2T2_nero_loss:
                return outputs, L2T2_nero_loss
            else:
                return outputs
        
        return self.base_model(*args, **kwargs)
    
    def __getattr__(self, name):
        try:
            return super().__getattr__(name) # Try getting attribute from self
        except AttributeError:
            return getattr(self.base_model, name) # Fallback to base_model

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, 
    device_map=device,
)
model = NeroModel(
    base_model, 
    L1T1_lora_config=model_configs['L1T1']['lora_config'], 
    L2T1_lora_config=model_configs['L2T1']['lora_config'], 
    train_L2T2_nero=False,
    return_L2T2_nero_loss=False,
    distance_fn='euclidean',
    distance_fn_pairwise=True,
    debug=False,
)

In [8]:
print("Check unloaded L1T1 LoRA parameters:")
check_lora_parameters(model, prefix='L1T1')
print()

model.load_lora_params('L1T1', model_configs['L1T1']['lora_path'])
print()

print("Check loaded L1T1 LoRA parameters:")
check_lora_parameters(model, prefix='L1T1')
print()

Check unloaded L1T1 LoRA parameters:
- name    : base_model.model.layers.0.self_attn.q_proj.lora.L1T1.A.weight
- device  : cuda:0
- dtype   : torch.float32
- mean    : 0.001683178823441267
- min     : -1.4880893230438232
- max     : 1.3589063882827759

[INFO] L1T1 LoRA parameters loaded successfully!

Check loaded L1T1 LoRA parameters:
- name    : base_model.model.layers.0.self_attn.q_proj.lora.L1T1.A.weight
- device  : cuda:0
- dtype   : torch.float32
- mean    : 6.287686119321734e-05
- min     : -0.04176201671361923
- max     : 0.04242725297808647



In [9]:
print("Check unloaded L2T1 LoRA parameters:")
check_lora_parameters(model, prefix='L2T1')
print()

model.load_lora_params('L2T1', model_configs['L2T1']['lora_path'])
print()

print("Check loaded L2T1 LoRA parameters:")
check_lora_parameters(model, prefix='L2T1')
print()

Check unloaded L2T1 LoRA parameters:
- name    : base_model.model.layers.0.self_attn.q_proj.lora.L2T1.A.weight
- device  : cuda:0
- dtype   : torch.float32
- mean    : -0.000640460173599422
- min     : -1.4666246175765991
- max     : 1.4599299430847168

[INFO] L2T1 LoRA parameters loaded successfully!

Check loaded L2T1 LoRA parameters:
- name    : base_model.model.layers.0.self_attn.q_proj.lora.L2T1.A.weight
- device  : cuda:0
- dtype   : torch.float32
- mean    : 2.2152138626552187e-05
- min     : -0.06327299773693085
- max     : 0.0625513345003128



In [10]:
model.freeze_all_except_L1T2_nero()
print()

# check_loss_and_grad_norm(nero_model, tokenizer)

In [11]:
model.gradient_checkpointing_enable({'use_reentrant': False})
print("[INFO] Gradient checkpointing enabled!")
print()

check_loss_and_grad_norm(model, tokenizer)

[INFO] Gradient checkpointing enabled!

Loss: tensor(3.4235, device='cuda:0', grad_fn=<NllLossBackward0>)
Gradient norm: 8.971054464635511


# Data

In [12]:
datasets = load_dataset(hf_data_id, data_dir=hf_data_dir)
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
})


In [13]:
train_loader = DataLoader(
    datasets['train'], 
    batch_size=batch_size, 
    shuffle=True, 
    collate_fn=default_data_collator,
)
print("[INFO] Total batches:", len(train_loader))

[INFO] Total batches: 1250


In [14]:
# Sanity check
first_batch = next(iter(train_loader))
print("First batch data shape (input_ids, attention_mask):")
print((
    first_batch['input_ids'].shape, 
    first_batch['attention_mask'].shape, 
))
print()

first_batch_text = tokenizer.batch_decode(first_batch['input_ids'], skip_special_tokens=True)[0]
print("First batch text:")
print(first_batch_text[:100], "...")
print()

check_loss_and_grad_norm(model, tokenizer)

First batch data shape (input_ids, attention_mask):
(torch.Size([4, 512]), torch.Size([4, 512]))

First batch text:
### Instruction:
Solve the following math problem step by step.

### Question: 
Lavinia’s daughter i ...

Loss: tensor(3.4235, device='cuda:0', grad_fn=<NllLossBackward0>)
Gradient norm: 8.971054464635511


# Training

In [15]:
# Sanity check
# Set model to learn L2T2 Nero
model.set_train_L2T2_nero(True)
model.set_distance_fn('huber', pairwise=True)
model.train()

# Forward pass
device = next(model.parameters()).device
# prompts = [
#     "The capital city of Japan is", 
#     "The capital city of Indonesia is",
# ]
# inputs = tokenizer(
#     prompts,
#     return_tensors='pt',
#     padding=True,
# )
outputs, L2T2_nero_outs = model(
    # Test with first batch
    input_ids=first_batch['input_ids'].to(device),
    attention_mask=first_batch['attention_mask'].to(device),
    labels=first_batch['input_ids'].to(device),

    # Test with sample prompts
    # input_ids=inputs['input_ids'].to(device),
    # attention_mask=inputs['attention_mask'].to(device),
    # labels=inputs['input_ids'].to(device),
    
    use_cache=False, # Disable cache to not conflict with gradient checkpointing
)

print(L2T2_nero_outs)

# pairwise euclidean: 0.2222
# non-pairwise euclidean: 0.2167

# pairwise cosine: 0.0022
# non-pairwise cosine: 0.0024

# pairwise squared euclidean: 33.0331
# non-pairwise squared euclidean: 33.1182

# pairwise huber: 6.0343
# non-pairwise huber: 6.2125

# pairwise cosine huber: 0.0640
# non-pairwise cosine huber: 0.0663

OutOfMemoryError: CUDA out of memory. Tried to allocate 8.00 GiB. GPU 0 has a total capacity of 14.74 GiB of which 7.34 GiB is free. Process 192539 has 7.40 GiB memory in use. Of the allocated memory 6.69 GiB is allocated by PyTorch, and 548.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [16]:
# Set the distance function
model.set_distance_fn(distance_fn, pairwise=True)

# Set the model to training mode
model.train()

# If `device` is not specified or set to 'auto', use the model's device
# if device is None or device == 'auto':
device = next(iter(model.parameters())).device

# Set up optimizer and gradient scaler
# for L2T2 Nero
model.set_train_L2T2_nero(True)
L2T2_nero_params = [p for n, p in model.named_parameters() if p.requires_grad]
L2T2_optimizer = torch.optim.Adam(L2T2_nero_params, lr=lr, foreach=True)
L2T2_scaler = torch.cuda.amp.GradScaler()

# for L1T2 Nero
model.set_train_L2T2_nero(False)
L1T2_nero_params = [p for n, p in model.named_parameters() if p.requires_grad]
L1T2_optimizer = torch.optim.Adam(L1T2_nero_params, lr=lr, foreach=True)
L1T2_scaler = torch.cuda.amp.GradScaler()

# Set up LR scheduler
max_global_steps = max_global_steps or len(train_loader) * num_epochs
warmup_steps = int(warmup_ratio * max_global_steps)
if warmup_ratio > 0:
    # If `warmup_ratio` > 0, use cosine annealing scheduler with warm-up 
    from transformers import get_cosine_schedule_with_warmup # type: ignore
    max_optimizer_steps = (max_global_steps // grad_accumulation_steps) // 2 # divide by 2 because we train in 2 modes
    num_warmup_steps = int(warmup_ratio * max_optimizer_steps)
    L1T2_scheduler = get_cosine_schedule_with_warmup(
        L1T2_optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=max_optimizer_steps,

        # Cycle length is 2000 steps or less, but at least 1 cycle, and at most 10 cycles
        num_cycles=min(max(1, max_optimizer_steps // 2000), 10), 
    )
    L2T2_scheduler = get_cosine_schedule_with_warmup(
        L2T2_optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=max_optimizer_steps,

        # Cycle length is 2000 steps or less, but at least 1 cycle, and at most 10 cycles
        num_cycles=min(max(1, max_optimizer_steps // 2000), 10), 
    )
else:
    # If `warmup_ratio` is 0, use a dummy scheduler that returns constant LR
    from torch.optim.lr_scheduler import LambdaLR # type: ignore
    L1T2_scheduler = LambdaLR(L1T2_optimizer, lr_lambda=lambda step: 1.0)
    L2T2_scheduler = LambdaLR(L2T2_optimizer, lr_lambda=lambda step: 1.0)

# Initialize W&B
wandb.init(
    project='Nero-XLT',
    reinit=True, # End previous run and start a new one
    config=dict(
        # Project configuration
        seed = seed,
        device = device,

        # Data configuration
        hf_data_id = hf_data_id,
        hf_data_dir = hf_data_dir,

        # Training configuration
        batch_size = batch_size,
        num_epochs = num_epochs,
        max_global_steps = max_global_steps,
        grad_accumulation_steps = grad_accumulation_steps,
        clip_grad_norm = clip_grad_norm,
        lr = lr,
        warmup_ratio = warmup_ratio,
        checkpoint_steps = checkpoint_steps,
        distance_fn = distance_fn,
        resume_step = resume_step,
    ),
)

# Resume training
global_step = resume_step
start_epoch = 0

def load_trainer_params(mode, model, optimizer, scheduler, scaler, checkpoint_dir, device):
    # Load Nero parameters
    nero_path = os.path.join(checkpoint_dir, f'{mode}_nero.safetensors')
    model.load_nero_params(mode, nero_path)

    # Load optimizer state
    optimizer_path = os.path.join(checkpoint_dir, f'{mode}_optimizer.pt')
    optimizer.load_state_dict(torch.load(optimizer_path, map_location=device))
    
    # Move optimizer state to the correct device
    for param in optimizer.state:
        param_device = param.device
        param_dtype = param.dtype
        for key, value in optimizer.state[param].items():
            if isinstance(value, torch.Tensor):
                optimizer.state[param][key] = value.to(device=param_device, dtype=param_dtype)

    # Load scheduler state
    scheduler_path = os.path.join(checkpoint_dir, f'{mode}_scheduler.pt')
    scheduler.load_state_dict(torch.load(scheduler_path, map_location=device))

    # Load scaler state
    scaler_path = os.path.join(checkpoint_dir, f'{mode}_scaler.pt')
    scaler.load_state_dict(torch.load(scaler_path, map_location=device))

if resume_step > 0:
    checkpoint_dir = os.path.join(nero_dir, f'checkpoint-{resume_step}')
    print(f"[INFO] Resuming training from checkpoint directory:", checkpoint_dir)

    # Load trainer parameters
    load_trainer_params('L1T2', model, L1T2_optimizer, L1T2_scheduler, L1T2_scaler, checkpoint_dir, device)
    load_trainer_params('L2T2', model, L2T2_optimizer, L2T2_scheduler, L2T2_scaler, checkpoint_dir, device)

    # Load trainer state
    trainer_state_path = os.path.join(checkpoint_dir, 'trainer_state.json')
    if os.path.exists(trainer_state_path):
        with open(trainer_state_path, 'r') as f:
            trainer_state = json.load(f)
        log_history = trainer_state.get('log_history', [])
        start_epoch = log_history[-1]['epoch'] if log_history else 0
        print(f"[INFO] Resuming training from epoch {start_epoch} and step {resume_step}.")

    # Load RNG state for reproducibility
    rng_path = os.path.join(checkpoint_dir, 'rng_state.pth')
    if os.path.exists(rng_path):
        rng_state = torch.load(rng_path)
        random.setstate(rng_state['python'])
        np.random.set_state(rng_state['numpy'])
        torch.set_rng_state(rng_state['cpu'])
        if torch.cuda.is_available() and rng_state['cuda']:
            torch.cuda.set_rng_state_all(rng_state['cuda'])
    
    if resume_step % grad_accumulation_steps != 0:
        print("[WARN] Resuming mid-gradient accumulation cycle. Make sure this is intended.")
else:
    if push_to_hf:
        # If it's new training, create Hugging Face repository
        print(f"[INFO] Creating Hugging Face repository:", hf_nero_id) # print the link instead
        create_repo(repo_id=hf_nero_id, repo_type='model', exist_ok=True)
        print(f"[INFO] Hugging Face repository created successfully!")

/tmp/ipykernel_1106/1621092812.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  L2T2_scaler = torch.cuda.amp.GradScaler()
/tmp/ipykernel_1106/1621092812.py:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  L1T2_scaler = torch.cuda.amp.GradScaler()
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alimtegar. Use `wandb login --relogin` to force relogin


In [17]:
train_L2T2_nero = False
log_history = []
done = False

# Safety: Zero gradients at the start of gradient accumulation cycle
# This ensures there are no leftover gradients when resuming mid-cycle or after a previous cycle was interrupted
if global_step % grad_accumulation_steps == 0:
    L1T2_optimizer.zero_grad(set_to_none=True)
    L2T2_optimizer.zero_grad(set_to_none=True)

# Training loop
for epoch in range(start_epoch, num_epochs):
    for step, batch in enumerate(train_loader):
        # Skip previously completed steps
        if global_step <= resume_step:
            global_step += 1
            continue

        # Stop training if `max_global_steps` reached
        if global_step >= max_global_steps:
            done = True
            break

        # Move inputs to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        with torch.cuda.amp.autocast():
            # Forward pass
            model.set_train_L2T2_nero(train_L2T2_nero)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=input_ids,
                use_cache=False, # Disable cache to avoid conflict with gradient checkpointing
            )
            
            if train_L2T2_nero:
                _, L2T2_nero_loss = outputs
                _loss = L2T2_nero_loss
            else:
                _loss = outputs.loss
                
            loss = _loss / grad_accumulation_steps

        log = {
            'mode': 'L1T2' if not train_L2T2_nero else 'L2T2',
            'epoch': epoch,
            'step': global_step,
        }

        # Backward pass
        with torch.autograd.set_detect_anomaly(True):
            if train_L2T2_nero:
                log['L2T2/loss'] = loss.item() * grad_accumulation_steps
                model.set_return_L2T2_nero_loss(False)
                L2T2_scaler.scale(loss).backward()
            else:
                log['L1T2/loss'] = loss.item() * grad_accumulation_steps
                print("loss:", loss)
                L1T2_scaler.scale(loss).backward()
        
        # Update parameters only at the end of gradient accumulation cycle
        grad_norm_log = {}
        if (global_step + 1) % grad_accumulation_steps == 0:
            if train_L2T2_nero:
                # Unscale gradients before computing gradient norm and applying clipping
                L2T2_scaler.unscale_(L2T2_optimizer)

                # Compute gradient norm
                grad_norm = compute_grad_norm(L2T2_nero_params)
                grad_norm_log['L2T2/grad_norm'] = grad_norm

                # Clip gradients
                if clip_grad_norm is not None:
                    torch.nn.utils.clip_grad_norm_(L2T2_nero_params, clip_grad_norm)
                
                # Compute clipped gradient norm
                grad_norm_clipped = compute_grad_norm(L2T2_nero_params)
                grad_norm_log['L2T2/grad_norm_clipped'] = grad_norm_clipped

                # Update parameters
                L2T2_scaler.step(L2T2_optimizer)
                L2T2_scaler.update()
                L2T2_scheduler.step()

                # Zero gradients for the next gradient accumulation cycle
                L2T2_optimizer.zero_grad(set_to_none=True)
            else:
                # Unscale gradients before computing gradient norm and applying clipping
                L1T2_scaler.unscale_(L1T2_optimizer)
                
                # Compute gradient norm
                grad_norm = compute_grad_norm(L1T2_nero_params)
                grad_norm_log['L1T2/grad_norm'] = grad_norm

                # Clip gradients
                if clip_grad_norm is not None:
                    torch.nn.utils.clip_grad_norm_(L1T2_nero_params, clip_grad_norm)
                
                # Compute clipped gradient norm
                grad_norm_clipped = compute_grad_norm(L1T2_nero_params)
                grad_norm_log['L1T2/grad_norm_clipped'] = grad_norm_clipped

                # Update parameters
                L1T2_scaler.step(L1T2_optimizer)
                L1T2_scaler.update()
                L1T2_scheduler.step()

                # Zero gradients for the next gradient accumulation cycle
                L1T2_optimizer.zero_grad(set_to_none=True)

            # After updating parameters, toggle training mode:
            # if currently training L1T2, switch to L2T2
            # if currently training L2T2, switch to L1T2
            train_L2T2_nero = not train_L2T2_nero

        # Logging
        lr_log = (
            {'L1T2/lr': L1T2_scheduler.get_last_lr()[0]} 
            if 'L1T2/loss' in log else 
            {'L2T2/lr': L2T2_scheduler.get_last_lr()[0]}
        )
        log = {
            **log, 
            **lr_log, 
            **grad_norm_log,
        }
        log_history.append(log)
        wandb.log(log)
        print(", ".join(
            f"{k}: {format_float(v)}" if isinstance(v, float) else f"{k}: {v}"
            for k, v in log.items()
        ))
        
        # Save and push checkpoint every `checkpoint_steps`
        if global_step > 0 and global_step % checkpoint_steps == 0:
            # Create checkpoint directory
            checkpoint_dir = os.path.join(nero_dir, f'checkpoint-{global_step}')
            os.makedirs(checkpoint_dir, exist_ok=True)

            # Save Nero parameters, along with optimizer, scheduler, and scaler states
            L1T2_nero_state_dict = {n: p.detach().cpu() for n, p in model.named_parameters() if 'L1T2' in n}
            save_file(L1T2_nero_state_dict, os.path.join(checkpoint_dir, 'L1T2_nero.safetensors'))
            torch.save(L1T2_optimizer.state_dict(), os.path.join(checkpoint_dir, 'L1T2_optimizer.pt'))
            torch.save(L1T2_scheduler.state_dict(), os.path.join(checkpoint_dir, 'L1T2_scheduler.pt'))
            torch.save(L1T2_scaler.state_dict(), os.path.join(checkpoint_dir, 'L1T2_scaler.pt'))

            L2T2_nero_state_dict = {n: p.detach().cpu() for n, p in model.named_parameters() if 'L2T2' in n}
            save_file(L2T2_nero_state_dict, os.path.join(checkpoint_dir, 'L2T2_nero.safetensors'))
            torch.save(L2T2_optimizer.state_dict(), os.path.join(checkpoint_dir, 'L2T2_optimizer.pt'))
            torch.save(L2T2_scheduler.state_dict(), os.path.join(checkpoint_dir, 'L2T2_scheduler.pt'))
            torch.save(L2T2_scaler.state_dict(), os.path.join(checkpoint_dir, 'L2T2_scaler.pt'))

            # Save trainer state for resuming training
            trainer_state = {
                'epoch': epoch,
                'global_step': global_step,
                'log_history': log_history,
            }
            with open(os.path.join(checkpoint_dir, 'trainer_state.json'), 'w') as f:
                json.dump(trainer_state, f, indent=2)

            # Save RNG state for reproducibility
            rng_state = {
                'python': random.getstate(),
                'numpy': np.random.get_state(),
                'cpu': torch.get_rng_state(),
                'cuda': torch.cuda.get_rng_state_all() if torch.cuda.is_available() else [],
            }
            torch.save(rng_state, os.path.join(checkpoint_dir, 'rng_state.pth'))

            # Upload checkpoint directory to Hugging Face repository
            if push_to_hf:
                upload_folder(
                    folder_path=checkpoint_dir,
                    repo_id=hf_nero_id,
                    path_in_repo=f"checkpoint-{global_step}",
                    commit_message=f"Add checkpoint at step {global_step}",
                    repo_type='model',
                )
        
        # Check generated text every `generate_steps`
        if global_step > 0 and global_step % generate_steps == 0:
            model.set_train_L2T2_nero(False)
            generated = generate_text(model, tokenizer, sample_prompt, device=device)
            print("================================")
            print("CHECK GENERATED TEXT")
            print("================================")
            print(f"{'Prompt':<9}:", sample_prompt)
            print(f"{'Generated':<9}:", generated)
            print()
        
        global_step += 1
    
    if done:
        break

wandb.finish()

/tmp/ipykernel_1106/3948051874.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


loss: tensor(3.2918, device='cuda:0', grad_fn=<DivBackward0>)


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


mode: L1T2, epoch: 0, step: 1, L1T2/loss: 6.5837, L1T2/lr: 6.4516e-06, L1T2/grad_norm: 2.8242, L1T2/grad_norm_clipped: 1.0000
mode: L2T2, epoch: 0, step: 2, L2T2/loss: 0.6633, L2T2/lr: 0.0000e+00
mode: L2T2, epoch: 0, step: 3, L2T2/loss: 0.6907, L2T2/lr: 6.4516e-06, L2T2/grad_norm: 0.1261, L2T2/grad_norm_clipped: 0.1261
loss: tensor(3.8685, device='cuda:0', grad_fn=<DivBackward0>)
mode: L1T2, epoch: 0, step: 4, L1T2/loss: 7.7371, L1T2/lr: 6.4516e-06
loss: tensor(3.8263, device='cuda:0', grad_fn=<DivBackward0>)


/opt/conda/lib/python3.10/site-packages/torch/autograd/graph.py:768: UserWarning: Error detected in MmBackward0. No forward pass information available. Enable detect anomaly during forward pass for more information. (Triggered internally at /usr/local/src/pytorch/torch/csrc/autograd/python_anomaly_mode.cpp:89.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


RuntimeError: Function 'MmBackward0' returned nan values in its 0th output.